<a href="https://colab.research.google.com/github/sidhu2690/CogniXis/blob/main/03_Computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip -O pizza_steak_sushi.zip
!unzip -q pizza_steak_sushi.zip -d ./data/

In [2]:
train_dir = "/content/data/train"
test_dir = "/content/data/test"

In [5]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [22]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std= [0.5, 0.5, 0.5])
])

In [23]:
train_data=datasets.ImageFolder(root=train_dir, transform= data_transforms)
test_data= datasets.ImageFolder(root=test_dir, transform= data_transforms)

In [24]:
train_loader=DataLoader(train_data, batch_size=32, shuffle= True)
test_loader= DataLoader(test_data, batch_size=32, shuffle= True)

In [25]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [26]:
class CNN_Model(nn.Module):

  def __init__(self):
    super().__init__()

    self.model= nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride= 1, padding=1),  #32, 224, 224
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                                     # 32, 112, 112

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),  # 64, 112, 112
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                                     # 64, 56, 56

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride= 1, padding=1),# 128, 56, 56
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                                    # 128, 28, 28

        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),# 256, 28, 28
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                                      # 256, 14, 14

        nn.Flatten(),
        nn.Linear(256* 14*14, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 3)

    )

  def forward(self, x):
    return self.model(x)

Model= CNN_Model().to(device)

In [27]:
loss_fn= nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=Model.parameters(),
                          lr=0.01)

In [28]:
def training_loop(model, train_loader, test_loader, optimizer, loss_fn, epochs):
    for epoch in range(epochs):
        model.train()

        train_loss, train_correct=0,0

        for X,y in train_loader:
            X,y= X.to(device), y.to(device)

            y_pred=model(X)
            loss=loss_fn(y_pred, y)

            train_loss+=loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_correct+= (y_pred.argmax(1) == y).sum().item()

        train_accuracy=train_correct / len(train_loader.dataset) * 100

        # evaluation

        model.eval()

        test_loss, test_correct= 0, 0

        with torch.no_grad():
            for X,y in test_loader:
                X, y= X.to(device), y.to(device)

                test_pred= model(X)
                loss=loss_fn(test_pred, y)
                test_loss+= loss.item()

                test_correct+= (test_pred.argmax(1) == y).sum().item()
            test_accuracy= test_correct / len(test_loader.dataset) * 100

            print(f"Epochs: {epoch} | train loss: {train_loss} | train accuracy: {train_accuracy} | test loss: {test_loss} | test accurcy: {test_accuracy}" )

training_loop(Model, train_loader, test_loader, optimizer, loss_fn, epochs=10)

Epochs: 0 | train loss: 8.787685871124268 | train accuracy: 32.0 | test loss: 3.287534236907959 | test accurcy: 41.333333333333336
Epochs: 1 | train loss: 8.813910245895386 | train accuracy: 32.0 | test loss: 3.294824957847595 | test accurcy: 41.333333333333336
Epochs: 2 | train loss: 8.785962343215942 | train accuracy: 33.77777777777778 | test loss: 3.301422357559204 | test accurcy: 25.333333333333336
Epochs: 3 | train loss: 8.801839828491211 | train accuracy: 35.11111111111111 | test loss: 3.300467610359192 | test accurcy: 25.333333333333336
Epochs: 4 | train loss: 8.78228211402893 | train accuracy: 35.55555555555556 | test loss: 3.2997387647628784 | test accurcy: 36.0
Epochs: 5 | train loss: 8.763275861740112 | train accuracy: 41.333333333333336 | test loss: 3.2996386289596558 | test accurcy: 33.33333333333333
Epochs: 6 | train loss: 8.809857964515686 | train accuracy: 35.11111111111111 | test loss: 3.2897286415100098 | test accurcy: 33.33333333333333
Epochs: 7 | train loss: 8.77907